In [29]:
import os
import glob
import string
import pandas as pd
import re
from nltk.tokenize import TweetTokenizer
from pyarabic.araby import strip_tashkeel,strip_tatweel,normalize_ligature
from transformers import AutoTokenizer, TFAutoModel
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from ipynb.fs.full.Data_fetching import fetch_data

In [1]:
def encoding(df): 
    unique_Dialect = df['dialect'].unique()
    label2id={label:idx for idx,label in enumerate(unique_Dialect)}
    id2label={idx:label for label,idx in label2id.items()}
    encoder= ce.OrdinalEncoder(cols=['dialect'],return_df=True,
                           mapping=[{'col':'dialect',
    'mapping':label2id}])
    df['transformed'] = encoder.fit_transform(df['dialect'])
    return df 
     

In [36]:
encoding(df)
df.head()

,id,dialect,transformed
0,1175358310087892992,IQ,0
1,1175416117793349632,IQ,0
2,1175450108898565888,IQ,0
3,1175471073770573824,IQ,0
4,1175496913145217024,IQ,0


In [34]:
# check for cleaning
df=pd.read_csv("dialect_dataset.csv",converters={'id': str})
df.head()
tweetdic=fetch_data(df)


In [37]:
df['text'] = df['id'].map(tweetdic)
df.head()

,id,dialect,transformed,text
0,1175358310087892992,IQ,0,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,0,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,0,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,0,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,0,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [39]:
def normalize(sent):
    """clean data from any English char, emoticons, underscore, and repeated > 2
    str -> str"""
    p1 = re.compile('\W')
    p2 = re.compile('\s+')
    sent = ReplaceThreeOrMore(sent)
    
    sent = sent.replace('_', ' ')
    sent = re.sub(r'[A-Za-z0-9]', r'', sent)
    
    sent = re.sub(p1, ' ', sent)
    
    sent = re.sub(p2, ' ', sent)
   
    return sent

def ReplaceThreeOrMore(s):
    # pattern to look for three or more repetitions of any character, including
    # newlines.
     
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL) #3 or more 
 
    return pattern.sub(r"\1", s)  



In [40]:
def remove_unicode_diac(text):
    """Takes Arabic in utf-8 and returns same text without diac"""
    # Replace diacritics with nothing
    text = text.replace(u"\u064B", "")  # fatHatayn
    text = text.replace(u"\u064C", "")  # Dammatayn
    text = text.replace(u"\u064D", "")  # kasratayn
    text = text.replace(u"\u064E", "")  # fatHa
    text = text.replace(u"\u064F", "")  # Damma
    text = text.replace(u"\u0650", "")  # kasra
    text = text.replace(u"\u0651", "")  # shaddah
    text = text.replace(u"\u0652", "")  # sukuun
    text = text.replace(u"\u0670", "`")  # dagger 'alif
    return text

def norm_alif(text):
    text = text.replace(u"\u0625", u"\u0627")  # HAMZA below, with LETTER ALEF 
    text = text.replace(u"\u0622", u"\u0627")  # ALEF WITH MADDA ABOVE, with LETTER ALEF
    text = text.replace(u"\u0623", u"\u0627")  # ALEF WITH HAMZA ABOVE, with LETTER ALEF
    return text

def norm_taa(text):
    text=text.replace(u"\u0629", u"\u0647") # taa' marbuuTa, with haa'
    
    return text
def norm_yaa(text):  
    if len(text)!=0:
        if text[-1] == u"\u064A":  
            s = list(text)
            s[-1]=u"\u0649"
            text="".join(s)
    #  text = text.replace(u"\u064A", u"\u0649")  # yaa' with 'alif maqSuura  
          
    return text 
def NormForWord2Vec(text):
    text=norm_taa(text)
    text=norm_yaa(text)
    text=norm_alif(text)
    return text

def remove_nonunicode2(Tweet):
    ## defining set of unicode ##
    #u""
    #Tweet=Tweet.decode("utf-8")
    UniLex={ ## This is list of all arabic unicode characters in addition to space (to separate words)
            u"\u0622",
            u"\u0626",
            u"\u0628",
            u"\u062a",
            u"\u062c",
            u"\u06af",
            u"\u062e",
            u"\u0630",
            u"\u0632",
            u"\u0634",
            u"\u0636",
            u"\u0638",
            u"\u063a",
            u"\u0640",
            u"\u0642",
            u"\u0644",
            u"\u0646",
            u"\u0648",
            u"\u064a",
            u"\u0670",
            u"\u067e",
            u"\u0686",
            u"\u0621",
            u"\u0623",
            u"\u0625",
            u"\u06a4",
            u"\u0627",
            u"\u0629",
            u"\u062b",
            u"\u062d",
            u"\u062f",
            u"\u0631",
            u"\u0633",
            u"\u0635",
            u"\u0637",
            u"\u0639",
            u"\u0641",
            u"\u0643",
            u"\u0645",
            u"\u0647",
            u"\u0649",
            u"\u0671",
            ' ',
            '\n'
          }
    fin_tweet=""
    for c in Tweet:
        if c in UniLex:
           fin_tweet=fin_tweet+c
    return fin_tweet


In [41]:
def eliminate_single_char_words(Tweet):
    parts = Tweet.split(" ")
    cleaned_line_parts = []
    for P in parts:
        if len(P) != 1:
            cleaned_line_parts.append(P)
    cleaned_line = ' '.join(cleaned_line_parts)
    return cleaned_line
def clean_unicode(Tweet):
    if(isinstance(Tweet, str)==False):
        print(Tweet)
    tweet=normalize(Tweet.strip("\n"))
    if len(tweet) !=0:
        sentence = []
        for word in tweet.split(" "):
            word = remove_unicode_diac(word)
            word = norm_alif(word)
            word = norm_taa(word)
            word = norm_yaa(word)
            word = normalize(word)
            sentence.append(word)
        tweet = ' '.join(sentence)
        tweet =remove_nonunicode2(tweet)
        tweet =eliminate_single_char_words(tweet)
    return tweet

In [43]:
df['fineText'] = df['text'].map(lambda x:clean_unicode(x) )

In [48]:
tempdf= df.copy()
del df['text']

In [55]:
df.to_csv('transformed.csv', index=False)

In [56]:
newdf=pd.read_csv("transformed.csv")
newdf.shape , df.shape

((458197, 4), (458197, 4))

In [54]:
newdf.head()

,Unnamed: 0,id,dialect,transformed,fineText
0,0,1175358310087892992,IQ,0,لكن بالنهايه ينتفض يغير
1,1,1175416117793349632,IQ,0,يعنى هذا محسوب على البشر حيونه ووحشيه وتطلبون...
2,2,1175450108898565888,IQ,0,مبين من كلامه خليجى
3,3,1175471073770573824,IQ,0,يسلملى مرورك وروحك الحلوه
4,4,1175496913145217024,IQ,0,وين هل الغيبه اخ محمد
